# Obbiettivi

Precedentemente ho stimato l'accuratezza degli alberi di decisione indotti da training set sottoposti ad augmentation con SMOTE o attraverso l'aggiunta di rumore casuale, per classificare i tumori nelle 5 categorie individuate (la 7 è stata eliminata dal ds e i tumori di classe 6 sono stati incorporati nella classe 1). I risultati non erano ottimali: l'accuratezza non superava il 20% e le regole che gli alberi utilizzavano per la classificaizione tenevano quasi solo conto di età e ADC.
Provo ora un approccio differente. Per ognuna delle 5 classi induco (usando augmentatiom, sia con smote che con il rumore casuale) un albero che classifica i samples come appartenenti a quella classe o meno (cioè realizza una classificazione binaria) e ne valuto le prestazioni. Infine darò le propabilità di classificazione fornite da ciascuno di questi alberi in pasto ad un ulteriore albero di decisione e ad una random forest di massimo 10 alberi, sempre con e senza data augmentation.


In [10]:
#importo il ds
import pandas as pd

df = pd.read_excel('Db_parotide_Def_REV_Samuel.xlsx')
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df['Output Algoritmo'].iloc[0] = 3
df['Output Algoritmo'].iloc[1] = 3
df['Output Algoritmo'].iloc[20] = 3
dfAlbero = df.iloc[:, 1:8]
dfAlbero = dfAlbero.drop(['Output Algoritmo'], axis = 1)
dfAlbero

/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,eta,Segni macro malignità,ADC,TIC type,T2,COD ISTO
0,33,0,2.60,D,0,1
1,50,0,2.44,D,0,1
2,81,1,0.70,C,1,1
3,28,0,1.60,A,1,1
4,51,0,1.40,A,0,1
...,...,...,...,...,...,...
102,27,0,1.30,C,1,1
103,50,0,1.90,A,0,5
104,62,0,2.10,A,0,5
105,37,0,1.80,A,0,5


In [11]:
#do un ulteriore sguardo alle classi minoritatrie nel ds
dfAlbero['COD ISTO'].value_counts()
#Penso di quadruplicarele 2 classi minoritarie col rumore random e con smotenc

3    54
4    27
1    16
2     5
5     5
Name: COD ISTO, dtype: int64

In [12]:
#creo i samples e le labels
X = dfAlbero.iloc[:,:-1]
Y = dfAlbero['COD ISTO']

## Innanzitutto rifinisco la classe che avevo implementato nel task precedente 

In [45]:
#per ogni cross val, l'augmentation è fatta solo sul training set, il test set è lasciato immacolato 
#ad ogni interazione
#usa KFold e non cross_val_score

#Questa classe serve per determinare l'accuratezza di un modello per la classificazione multi classe
#rispetto a dei dati 
#agumentanti con smote o col rumore random

import numpy as np
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from collections import Counter
import random
import pandas as pd

class SEC_aug(object):
    def __init__(self, X, Y, aug_method, learning_alg, n_fold, n_it, seed):
        self.X = X
        self.Y = Y
        self.aug_method = aug_method
        self.learning_alg = learning_alg
        self.n_fold = n_fold
        self.n_it = n_it
        self.seed = seed
        self.accuracy_median_scores = []
    #oversamplo tutte le classi tranne quella maggioritataria duplicando samples scelti randomicamente
    def random_oversample_raw(self, X_train, Y_train):
        ros = RandomOverSampler(random_state = self.seed)
        X_train_aug, Y_train_aug = ros.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #oversample solo le classi specificate in diz, secondo quanto specificato in diz
    def random_oversample_dict(self, X_train, Y_train, diz):
        ros = RandomOverSampler(sampling_strategy = diz, random_state = self.seed)
        X_train_aug, Y_train_aug = ros.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #oversamplo tutte le classi non maggioritatarie con SMOTE-->solo feature numeriche!!
    #Ho dovuto ridurre il k_neighbors a 2 perchè nei training set infividuati con la cross val
    #talvolta, sopratutto per le due classi minoritarie 2 e 5, vi è un numero limitato di samples
    #In ogni caso ho imposto nella cross validation di fare augmentation e calcolo accuratezza
    #solo negli split in cui il training set contiene almeno 2 rappresentanti per la classe 2 e 5
    #in quanto ritengo che avendo a diposizione solo 1 o meno samples da cui fare interpolazione 
    #non porti ad una buona augmentation
    
    #ciò significa anche che dovrò fare un elevato numero di iterazioni
    def SMOTE_augmentation_raw(self, X_train, Y_train):
        smt = SMOTE(random_state = self.seed , k_neighbors = 1)#questo perchè in alcuni split vi sono pochissimi rappresentanti di certe classi
        X_train_aug, Y_train_aug = smt.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #oversamplo tutte le classi specificate nel dizionario-->solo feature numeriche!!
    def SMOTE_augmentation_dict(self, X_train, Y_train, diz):
        smt = SMOTE(sampling_strategy = diz, random_state = self.seed , k_neighbors = 1)# a causa di scarso numero rappresentanti delle classi minoritarie nelle fold
        X_train_aug, Y_train_aug = smt.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #anche qua resamplo tutte le classi-->per feature sia chategoriche che numeriche!!
    def SMOTENC_augmentation_raw(self, X_train, Y_train, cat_indexes):#assumo di non aver ancora fatto one-hot-encoding
        smt = SMOTENC(k_neighbors = 1, random_state=self.seed, categorical_features=cat_indexes)
        X_train_aug, Y_train_aug = smt.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #anche qua resamplo le classi indicate dal dizionario-->per feature sia categoriche che numeriche!!
    def SMOTENC_augmentation_dict(self, X_train, Y_train, cat_indexes, diz):#assumo di non aver ancora fatto one-hot-encoding
        smt = SMOTENC(k_neighbors = 2, random_state=self.seed, categorical_features=cat_indexes, sampling_strategy = diz)
        X_train_aug, Y_train_aug = smt.fit_resample(X_train, Y_train)
        return X_train_aug, Y_train_aug
    
    #oversampling con aggiunta rumore random-->quadruplico i samples delle sole 2 classi minoritarie
    #err_perc --> la probabilità che un sample artificiale venga sottoposto ad aggiunta di rumore
    #prob_modifica --> la probabilità che il valore di una feature di un sample artificiale a cui
    #applicare rumore venga modificato
    def oversample_augmentation(self, X_train, Y_train, err_perc, prob_modifica):
        
        if err_perc > 1:
            err_perc /= 100
        
        #ricavo range di ciascuna feature numerica
        min_features = X_train.drop(['TIC type'], axis = 1).min()
        max_features = X_train.drop(['TIC type'], axis = 1).max()
        
        #metà del range per ciascuna feature numerica
        scales = (max_features-min_features)/2
        
        #numero di samples iniziali per ogni classe
        n_samples_iniziali = Counter(Y_train)
        
        #indice dal quale sono aggiunti i samples artificiali
        index_partenza_art = len(Y_train)
        
        #pareggio le  classi introducendo nuovi samples artificiali
        
        diz = {2: n_samples_iniziali[2]*4, 5: n_samples_iniziali[5]*4}#quadruplo le 2 classi minoritarie
        #per il momento il ratio è hard coded, ma potrei modificare la funzione per adattarla 
        #anche a ds diversi. Per ora non ne sento la necessità, devo solo triplicare due classi specifiche
        X_train_aug, Y_train_aug = self.random_oversample_dict(X_train, Y_train, diz)
        
        #salvo numero di samples aggiunti per ogni classe
        n_samples_post_aug = Counter(Y_train_aug)
        added_samples = n_samples_post_aug-n_samples_iniziali
        
        #calcolo numero di samples artificiali a cui applicherò del rumore, in funzione di err_perc
        n_apply_noise = dict()
        for k in added_samples.keys():
             n_apply_noise[k] = round(added_samples[k] * err_perc)
                
        #ricavo gli indici dei samples artificiali delle varie classi a cui devo applicare rumore                                                     
        
        indexes_2 = random.sample([index for index in Y_train_aug.index if Y_train_aug[index] == 2 and index >= index_partenza_art], n_apply_noise[2])
        indexes_5 = random.sample([index for index in Y_train_aug.index if Y_train_aug[index] == 5 and index >= index_partenza_art], n_apply_noise[5])
       
        indici_da_modificare = pd.Series([indexes_2,indexes_5])
        indici_da_modificare.reindex([2,5])
        #sempre hard coded, ma per ora mi va bene
     
        mu,sigma=0, 0.1
        
        
        for classe, indici in enumerate(indici_da_modificare):
            for indice in indici:
                for feature in X_train_aug.columns:
                    n = random.uniform(0,1)
                    if n < prob_modifica:
                        if feature.startswith('TIC'):#la feature è categorica
                            X_train_aug.at[indice, feature] = random.choice(X_train_aug['TIC type'].unique())
                        else:#la feature è numerica
                            X_train_aug.at[indice, feature] = round(X_train_aug.at[indice, feature]+scales[feature]*np.random.normal(mu, sigma))
                            #verifico che se fosse uscito dall'intervallo gli imposto il valore massimo                                        
                            X_train_aug.at[indice, feature]=min(X_train_aug.at[indice, feature], max_features[feature])
                            #verifico che se fosse uscito dall'intervallo gli imposto il valore minimo
                            X_train_aug.at[indice, feature]=max(X_train_aug.at[indice, feature], min_features[feature])

        return X_train_aug, Y_train_aug
    
    #prima crea le folder per cross-val, poi augmenta i sample del training set, poi fitta il modello e poi calcola accuratezza
    def fit_score(self):
        for niter in range(self.n_it):
            #e setto che fa shuffling
            #random_state = self.seed rimosso, perchè credo che se lo setto ad ogni iterazione dà lo stesso split
            #ogni volta, invece io vorrei avere folder diverse ad ogni interazione
            #Potrei pensare anche di usare RepeatedKFold
            
            sf = KFold(n_splits = self.n_fold, shuffle = True)
            
            accuracy_scores = []
           
            for train_index, test_index in sf.split(self.X, self.Y):
                X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
                Y_train, Y_test = self.Y.iloc[train_index], self.Y.iloc[test_index]
                
                
                if Y_train.value_counts()[2] <= 1 or Y_train.value_counts()[5] <= 1:#in caso contrario non è possibile applicare SMOTE
                    continue
    
                diz = dict()#anche questo è hard coded
                diz[2] = 4*len([x for x in Y_train if x == 2])
                diz[5] = 4*len([x for x in Y_train if x == 5])
                
            
                #Augmentation
                #Anche qua per ora ho degli elementi hard coded
                #Sostanzialmente faccio che o oversamplo tutte le classi tranne la maggioritaria (raw)
                #oppure quadruplico le due classi minoritarie nel ds (non raw)
                if self.aug_method == 'smote':
                    X_train_aug, Y_train_aug = self.SMOTE_augmentation_dict(X_train, Y_train, diz)
               
                elif self.aug_method == 'smote_raw':
                    X_train_aug, Y_train_aug = self.SMOTE_augmentation_raw(X_train, Y_train)
 
                elif self.aug_method == 'smotenc':
                    X_train_aug, Y_train_aug = self.SMOTENC_augmentation_dict(X_train, Y_train, [3], diz)

                elif self.aug_method == 'smotenc_raw':
                    X_train_aug, Y_train_aug = self.SMOTENC_augmentation_raw(X_train, Y_train, [3])
 
                elif self.aug_method == 'oversample':#ci sono due parametri hard coded
                    X_train_aug, Y_train_aug = self.oversample_augmentation(X_train, Y_train,err_perc=75, prob_modifica=0.25)

                else:#no data augmentation
                    X_train_aug, Y_train_aug = X_train, Y_train
                    
                X_train_aug = pd.get_dummies(X_train_aug, columns = ['TIC type'])
                #Training
                
                #Per gestire casi in cui nel training set non siano presenti sample con certi tipi di TIC
                columns = X_train_aug.columns
                if('TIC type_A' not in columns):
                    X_train_aug['TIC type_A'] = 0
                if('TIC type_B'not in columns):
                    X_train_aug['TIC type_B']= 0
                if('TIC type_C'not in columns):
                    X_train_aug['TIC type_C'] = 0
                if('TIC type_D'not in columns):
                    X_train_aug['TIC type_D'] =0
                
                
                clf = self.learning_alg.fit(X_train_aug, Y_train_aug)
                
                #Predictions
                #Per gestire il fatto che nel test set potrebbero non esserci samples con una certa curva TIC
                X_test = pd.get_dummies(X_test, columns = ['TIC type'])
                columns = X_test.columns
                if('TIC type_A' not in columns):
                    X_test['TIC type_A'] = 0
                if('TIC type_B'not in columns):
                    X_test['TIC type_B']= 0
                if('TIC type_C'not in columns):
                    X_test['TIC type_C'] = 0
                if('TIC type_D'not in columns):
                    X_test['TIC type_D'] =0
                    
                    
                predictions = clf.predict(X_test)
                
                #calcolo accuracy
                accuracy_scores.append(accuracy_score(predictions, Y_test))
                
                
            self.accuracy_median_scores.append( np.mean(accuracy_scores))
            
    def print_result(self):
        print("Accuratezza: %f\n" %(np.mean(self.accuracy_median_scores)))
        self.accuracy_median_scores = []


In [48]:
#OK, la classe per l'augmentation per la classificazione multi classe è stata definita
#Faccio una prova

#Accuratezza usando l'introduzione di rumore random per l'augmentation
from sklearn import tree
import graphviz

seed = 42

alberoProva = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_aug(X,Y, 'oversample', alberoProva, 5, 50, seed )
prova.fit_score()
prova.print_result()

Accuratezza: 0.315506



In [49]:
#Accuratezza usando smotenc dove quadruplo solo le 2 classi minoritarie

alberoProva = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_aug(X,Y, 'smotenc', alberoProva, 5, 50, seed )
prova.fit_score()
prova.print_result()

Accuratezza: 0.331423



In [50]:
#Accuratezza usando smotenc_raw dove oversamplo tutte le classi non minoratarie

alberoProva = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_aug(X,Y, 'smotenc_raw', alberoProva, 5, 50, seed )
prova.fit_score()
prova.print_result()

Accuratezza: 0.252530



## Ora induco gli alberi per le classificazioni binarie sopra menzionate

In [51]:
#Albero per la classificazione 1 vs 2,3,4,5

#1 indica che appartiene alla classe specificata
#0 indica che non vi appartiene

dfAlbero1 = dfAlbero.copy()
Y = [1 if x == 1 else 0 for x in dfAlbero1['COD ISTO'] ]
X = dfAlbero1.iloc[:, :-1]


X_encoded = pd.get_dummies(X, columns = ['TIC type'])
X_encoded

,eta,Segni macro malignità,ADC,T2,TIC type_A,TIC type_B,TIC type_C,TIC type_D
0,33,0,2.60,0,0,0,0,1
1,50,0,2.44,0,0,0,0,1
2,81,1,0.70,1,0,0,1,0
3,28,0,1.60,1,1,0,0,0
4,51,0,1.40,0,1,0,0,0
...,...,...,...,...,...,...,...,...
102,27,0,1.30,1,0,0,1,0
103,50,0,1.90,0,1,0,0,0
104,62,0,2.10,0,1,0,0,0
105,37,0,1.80,0,1,0,0,0


In [52]:

#ho 16 samples della classe 1, quindi magari l'oversample non è necessario per avere un buon albero
#faccio una cv a 5 fold veloce sull'intero ds non augmentato
from sklearn import tree
from sklearn.model_selection import cross_val_score
albero1 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
scores = cross_val_score(albero1, X_encoded, Y)
print("Accuratezza: %f\n" %scores.mean())

Accuratezza: 0.832035



In [53]:
#Raccolgo qualche dato in più
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import classification_report

X_train, X_test, Y_train, Y_test = train_test_split(X_encoded,Y, test_size = 0.2, random_state = seed)
albero1 = albero1.fit(X_train, Y_train)
predictions = albero1.predict(X_test)

print(predictions)
print(classification_report(Y_test, predictions, digits = 3))

#Non va bene, l'albero classifica tutti gli input come non 1 e l'accuratezza è elevata a causa di pochi samples 1
#L'augmentation SERVE dunque

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0      0.773     1.000     0.872        17
           1      0.000     0.000     0.000         5

    accuracy                          0.773        22
   macro avg      0.386     0.500     0.436        22
weighted avg      0.597     0.773     0.674        22



/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [94]:
#Classe per l'Augmentation per le classificazioni binarie
#Qua l'oversampling porta il numero di samples della classe minoritaria al numero di samples di 
#quella maggioritaria

from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, recall_score, precision_score, matthews_corrcoef, cohen_kappa_score
from sklearn import metrics
import numpy as np
from collections import Counter
import random

class SEC_bin(object):
    def __init__(self, X, Y, data_aug_method, learning_alg, n_fold, n_it, seed):
        self.X = X
        self.Y=Y
        self.data_aug_method = data_aug_method
        self.learning_alg = learning_alg
        self.n_fold = n_fold
        self.n_it = n_it
        self.seed = seed
        
        self.spec=np.zeros(self.n_it)
        self.sens = np.zeros(self.n_it)
        self.f1=np.zeros(self.n_it)
        self.acc=np.zeros(self.n_it)
        self.prec=np.zeros(self.n_it)
        self.matt =np.zeros(self.n_it)
        self.cohen=np.zeros(self.n_it)  

    #data augmentation se non sono presenti variabili categoriche (Quindi usa SMOTE)
    def SMOTE_augmentation(self, X_train, Y_train):
    
        sm=SMOTE(random_state=self.seed)
        X_train_aug, Y_train_aug = sm.fit_resample(X_train, Y_train)

        return X_train_aug, Y_train_aug
    
    #data augmentation se sono presenti variabili categoriche (Quindi usa SMOTENC)
    def SMOTENC_augmentation_cat(self, X_train, Y_train, cat_indexes):
    
        sm=SMOTENC(random_state=self.seed, categorical_features=cat_indexes, k_neighbors = 5)
        X_train_aug, Y_train_aug = sm.fit_resample(X_train, Y_train)

        return X_train_aug, Y_train_aug
    
    #data augmentation con oversample
    def oversample_augmentation(self, X_train, Y_train, err_perc, prob_modifica): #err_perc è la percentuale di quelli che avranno l'errore nella data aug.
        #conta quante volte compare 1 e quante 0 nell'elemtno passato tra ().
        counter = Counter(Y_train) 
   
        #così so quanti sono gli elementi a cui dovrò mettere un errore.
        missing=counter[0]-counter[1]
 
        #creo un indice da cui so che i successivi elementi sono stati aggiunti
        partenza=counter[0]+counter[1] 
   
        #prendo il valore min e max per ogni feature numerica
        min_feat = X_train.drop(['TIC type'], axis = 1).min()
        max_feat = X_train.drop(['TIC type'], axis = 1).max()
        
        #prendo la metà dell'intervallo per ogni feat. numerica. Se mod_prob=1 allora shifto di mezzo intervallo.
        scales=(max_feat-min_feat)/2 
        
        #verifico la percentuale inserita nell'argomento
        if  err_perc>1:
            err_perc=err_perc/100
        
        #pareggio le due classi
        oversample = RandomOverSampler(sampling_strategy='minority', random_state = self.seed)
    
        X_train_aug, Y_train_aug = oversample.fit_resample(X_train, Y_train)
        
        #numero di quelli che avranno l'errore modificare (quelli che voglio aggiungere*%). Potrebbe non essere intero
        n_modificate=missing*err_perc
        #ora è intero: round approssima all'intero più vicino
        n_modificate=round(n_modificate) 
        #andrò a prendere n_modificate alla fine di X_train_selected_aug, andando a vedere le ultime "missing".
        
#mi salvo gli indici dei valori a cui devo andare ad aggiugere l'errore (l'errore viene aggiunto solo ai valori "nuovi")
        indici_prescelti= random.sample(range(partenza, partenza+missing), n_modificate)
        
        
        features = X_train_aug.columns                                                                                

        mu,sigma=0, 0.1
    #itero indici e features. E vado poi a verificare quanto vale prob_mod per vedere se il valore viene modificato e, se si, di quanto
    #quindi scorro le righe e dopo le colonne    
        features = X_train_aug.columns
        for indice in indici_prescelti:
            
            for feature in features:
                
                #tre casi per le possibili feature che posso trovare
                if abs(random.uniform(0,1))< prob_modifica:
                    if feature.startswith('TIC'):
                        
                        #vado a calcolare il modulo della probabilità di modifica: se è >sigma (68%?) allora ne inverto il valore
                        X_train_aug.at[indice, feature] = random.choice(X_train_aug['TIC type'].unique())
               
                    else :
                        X_train_aug.at[indice, feature] = round(X_train_aug.at[indice, feature]+scales[feature]*np.random.normal(mu, sigma))
                        #verifico che se fosse uscito dall'intervallo gli imposto il valore massimo                                        
                        X_train_aug.at[indice, feature]=min(X_train_aug.at[indice, feature], max_feat[feature])
                        #verifico che se fosse uscito dall'intervallo gli imposto il valore minimo
                        X_train_aug.at[indice, feature]=max(X_train_aug.at[indice, feature], min_feat[feature])


        return X_train_aug, Y_train_aug

    def fit_score(self):
        y_pred=np.zeros(self.Y.shape[0])
        
        #n iterazioni
        for niter in range(self.n_it):
            
            skf = StratifiedKFold(n_splits=self.n_fold)
            
            #outerfold
            for train_index, test_index in skf.split(self.X, self.Y): 
                                    
                X_train, X_test = self.X.iloc[train_index,:], self.X.iloc[test_index,:]

                Y_train, Y_test = self.Y.iloc[train_index], self.Y.iloc[test_index]
                
# DATA AUGMENTATION
                
                if self.data_aug_method=='smote':
        
                    #array che viene creato ogni volta per 
                    cat_indexes = [3]#hard coded
                    #divido in categorici e numerici. i lo uso come indice, feature mi indica la i-esima feature 
                   
                    if cat_indexes:
                        X_train_aug, Y_train_aug = self.SMOTENC_augmentation_cat(X_train, Y_train, cat_indexes)
                      
                    else:
                        X_train_aug, Y_train_aug = self.SMOTE_augmentation(X_train, Y_train)
                      
                elif self.data_aug_method=='oversample':                                                                                                           
                    X_train_aug, Y_train_aug = self.oversample_augmentation(X_train, Y_train, err_perc=75, prob_modifica=0.25)
                else:
                    X_train_aug, Y_train_aug = X_train, Y_train
                    #return 
##TRAIN
                X_train_aug = pd.get_dummies(X_train_aug, columns = ['TIC type'])
                #Training
                columns = X_train_aug.columns
                if('TIC type_A' not in columns):
                    X_train_aug['TIC type_A'] = 0
                if('TIC type_B'not in columns):
                    X_train_aug['TIC type_B']= 0
                if('TIC type_C'not in columns):
                    X_train_aug['TIC type_C'] = 0
                if('TIC type_D'not in columns):
                    X_train_aug['TIC type_D'] =0
                
                
                clf = self.learning_alg.fit(X_train_aug, Y_train_aug)
    
##TEST
                X_test = pd.get_dummies(X_test, columns = ['TIC type'])
                columns = X_test.columns
                if('TIC type_A' not in columns):
                    X_test['TIC type_A'] = 0
                if('TIC type_B'not in columns):
                    X_test['TIC type_B']= 0
                if('TIC type_C'not in columns):
                    X_test['TIC type_C'] = 0
                if('TIC type_D'not in columns):
                    X_test['TIC type_D'] =0
                    
                y_pred[test_index] = clf.predict(X_test)
              
        #fine ciclo for train_index, test_index in skf.split(X, Y):

        # QUI ho le predizioni ottenute con questo split del dataset
        #mi salvo il valore di ogni indice statistico, nell'array. Dovrò farne la media.
            self.f1[niter]=f1_score(y_pred,self.Y)
            self.sens[niter] = recall_score(y_pred,self.Y)
            self.acc[niter]=accuracy_score(y_pred,self.Y)
            self.prec[niter]=precision_score(y_pred,self.Y)
            self.matt[niter]=matthews_corrcoef(y_pred,self.Y)
            self.cohen[niter]=cohen_kappa_score(y_pred,self.Y)
            

    def print_result(self):
        print("f1: %f\n" %(np.mean(self.f1)))
        print("Sensibilità: %f\n" %(np.mean(self.sens)))
        print("Accuratezza: %f\n" %(np.mean(self.acc)))
        print("Precisione: %f\n" %(np.mean(self.prec)))
        print("Matthews: %f\n" %(np.mean(self.matt)))
        print("Kappa di Cohen: %f\n" %(np.mean(self.cohen)))
   





# Induzione alberi per le classificazioni binarie
## Albero 1 vs Altri 

In [66]:
#Ora che ho definito la classe per l'augmentation nella classificazione binaria
#osservo come le performance cambiano utilizzando data augmentation smote
seed = 42

In [67]:
#1 vs altri con smote
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 1 else 0 for x in dfAlbero['COD ISTO']])

albero1 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'smote', albero1, 5, 50, seed)
prova.fit_score()
prova.print_result()



f1: 0.295142

Sensibilità: 0.222702

Accuratezza: 0.687477

Precisione: 0.437500

Matthews: 0.132281

Kappa di Cohen: 0.120911



In [68]:
#Ora 1 vs Altri con il rumore random (75,25)
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 1 else 0 for x in dfAlbero['COD ISTO']])

albero1 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'oversample', albero1, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.244438

Sensibilità: 0.187533

Accuratezza: 0.672897

Precisione: 0.353750

Matthews: 0.065978

Kappa di Cohen: 0.061075



### Probabilità 1 vs altri

In [69]:
#I risultati sembrano comparabili
#induco ora l'albero 1 vs 2,3,4,5 allenato su tutto il ds facendo augmentation con smote(nc)
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 1 else 0 for x in dfAlbero['COD ISTO']])
albero1 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

sm=SMOTENC(random_state=seed, categorical_features=[3])#k_neighbors lasciato a default (5)
X_aug, Y_aug = sm.fit_resample(X, Y)
X_aug = pd.get_dummies(X_aug, columns = ['TIC type'])
X = pd.get_dummies(X, columns = ['TIC type'])
albero1 = albero1.fit(X_aug, Y_aug)
predictions = albero1.predict(X)
print(classification_report(Y, predictions, digits = 3))


              precision    recall  f1-score   support

           0      0.892     1.000     0.943        91
           1      1.000     0.312     0.476        16

    accuracy                          0.897       107
   macro avg      0.946     0.656     0.710       107
weighted avg      0.908     0.897     0.873       107



In [70]:
#Non male
#ora mi prendo le probabilità 1 vs altri
prob1 = albero1.predict_proba(X)
prob1

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.54545455, 0.45454545],
       [0.65151515, 0.34848485],
       [0.65151515, 0.34848485],
       [0.54545455, 0.45454545],
       [0.7       , 0.3       ],
       [0.54545455, 0.45454545],
       [0.        , 1.        ],
       [0.54545455, 0.45454545],
       [0.65151515, 0.34848485],
       [0.54545455, 0.45454545],
       [0.7       , 0.3       ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.95238095, 0.04761905],
       [0.65151515, 0.34848485],
       [0.65151515, 0.34848485],
       [0.65151515, 0.34848485],
       [0.54545455, 0.45454545],
       [0.7       , 0.3       ],
       [0.95238095, 0.04761905],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.95238095, 0.04761905],
       [0.95238095, 0.04761905],
       [0.65151515, 0.34848485],
       [0.65151515, 0.34848485],
       [0.54545455, 0.45454545],
       [0.65151515, 0.34848485],
       [0.

## Albero 2 vs Altri

In [72]:
#Ora valuto anche se per 2 vs altri smote e rumore random sono metodi di oversampling comparabili
#Ho dovuto ridurre il parametro k_neighbors a 3 a causa di scarso numero rappresentanti

X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 2 else 0 for x in dfAlbero['COD ISTO']])

albero2 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'smote', albero2, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.055556

Sensibilità: 0.032258

Accuratezza: 0.682243

Precisione: 0.200000

Matthews: -0.043789

Kappa di Cohen: -0.027103



In [73]:
#ora provo random noise
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 2 else 0 for x in dfAlbero['COD ISTO']])
albero2 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'oversample', albero2, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.031013

Sensibilità: 0.019127

Accuratezza: 0.756449

Precisione: 0.084000

Matthews: -0.066214

Kappa di Cohen: -0.048455



In [ ]:
#random noise qua sembra essere leggermente più accurato, ma per consistenza uso smote

### Probabilità 2 vs Altri

In [74]:

#2 vs altri
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 2 else 0 for x in dfAlbero['COD ISTO']])

albero2 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

sm=SMOTENC(random_state=seed, categorical_features=[3], k_neighbors = 4)#k_neighbors abbassato per pochi rappresentanti
X_aug, Y_aug = sm.fit_resample(X, Y)

X_aug = pd.get_dummies(X_aug, columns = ['TIC type'])
X = pd.get_dummies(X, columns = ['TIC type'])
albero2 = albero2.fit(X_aug, Y_aug)
predictions = albero2.predict(X)
print(classification_report(Y, predictions, digits = 3))


              precision    recall  f1-score   support

           0      0.973     0.716     0.825       102
           1      0.094     0.600     0.162         5

    accuracy                          0.710       107
   macro avg      0.534     0.658     0.494       107
weighted avg      0.932     0.710     0.794       107



In [76]:
prob2 = albero2.predict_proba(X)
prob2

array([[0.66666667, 0.33333333],
       [0.66666667, 0.33333333],
       [1.        , 0.        ],
       [0.38181818, 0.61818182],
       [0.38181818, 0.61818182],
       [1.        , 0.        ],
       [0.10810811, 0.89189189],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.66666667, 0.33333333],
       [0.38181818, 0.61818182],
       [0.10810811, 0.89189189],
       [0.10810811, 0.89189189],
       [0.5       , 0.5       ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.

## Albero 3 vs Altri

In [77]:
#ora valuto smote vs random noise per la classificazione dei tumori classe 3

#classe 3 è maggioritaria-->costituisce metà dataset-->provo solo augmentation con smote

X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 3 else 0 for x in dfAlbero['COD ISTO']])

albero3 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'smote', albero3, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.425532

Sensibilità: 0.500000

Accuratezza: 0.495327

Precisione: 0.370370

Matthews: -0.007222

Kappa di Cohen: -0.006971



In [78]:
#provo senza augmentation
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 3 else 0 for x in dfAlbero['COD ISTO']])
albero3 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'nulla', albero3, 5, 50, seed)
prova.fit_score()
prova.print_result()

#poca differenza tra smote e nessuna agmentation

f1: 0.470588

Sensibilità: 0.500000

Accuratezza: 0.495327

Precisione: 0.444444

Matthews: -0.008430

Kappa di Cohen: -0.008377



### Probabilità 3 vs Altri

In [79]:
#3 vs altri
#per consistenza anche qua applico smote alla classe minoritaria(in questo caso 0)
X = dfAlbero.iloc[:, :-1].copy()
Y = pd.Series([1 if x == 3 else 0 for x in dfAlbero['COD ISTO']])


albero3 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

sm=SMOTENC(random_state=seed, categorical_features=[3])#k_neighbors lasciato a default (5)
X_aug, Y_aug = sm.fit_resample(X, Y)

X_aug = pd.get_dummies(X_aug, columns = ['TIC type'])
X = pd.get_dummies(X, columns = ['TIC type'])
albero3 = albero3.fit(X_aug, Y_aug)
predictions = albero3.predict(X)
print(classification_report(Y, predictions, digits = 3))

              precision    recall  f1-score   support

           0      0.711     0.604     0.653        53
           1      0.661     0.759     0.707        54

    accuracy                          0.682       107
   macro avg      0.686     0.682     0.680       107
weighted avg      0.686     0.682     0.680       107



In [80]:
prob3 = albero3.predict_proba(X)
prob3


array([[0.54545455, 0.45454545],
       [0.42857143, 0.57142857],
       [0.48      , 0.52      ],
       [0.54545455, 0.45454545],
       [0.42857143, 0.57142857],
       [0.66666667, 0.33333333],
       [0.66666667, 0.33333333],
       [0.48      , 0.52      ],
       [0.66666667, 0.33333333],
       [0.48      , 0.52      ],
       [0.48      , 0.52      ],
       [0.48      , 0.52      ],
       [0.66666667, 0.33333333],
       [1.        , 0.        ],
       [0.48      , 0.52      ],
       [0.66666667, 0.33333333],
       [0.54545455, 0.45454545],
       [0.66666667, 0.33333333],
       [0.42857143, 0.57142857],
       [0.48      , 0.52      ],
       [0.66666667, 0.33333333],
       [0.48      , 0.52      ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.5       , 0.5       ],
       [0.48      , 0.52      ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.48      , 0.52      ],
       [0.        , 1.        ],
       [0.

### Albero 4 vs Altri

In [81]:
#ora valuto 4 vs altri smote
X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 4 else 0 for x in dfAlbero['COD ISTO']])


albero4 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'smote', albero4, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.246628

Sensibilità: 0.214198

Accuratezza: 0.551776

Precisione: 0.291111

Matthews: -0.063324

Kappa di Cohen: -0.061815



In [82]:
#ora provo random noise 
X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 4 else 0 for x in dfAlbero['COD ISTO']])

albero4 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'oversample', albero4, 5, 50, seed)
prova.fit_score()
prova.print_result()

f1: 0.181682

Sensibilità: 0.167151

Accuratezza: 0.548972

Precisione: 0.202963

Matthews: -0.125741

Kappa di Cohen: -0.124306



### Probabilità 4 vs Altri


In [83]:
#4 vs altri
X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 4 else 0 for x in dfAlbero['COD ISTO']])


albero4 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

sm=SMOTENC(random_state=seed, categorical_features=[3])#k_neighbors lasciato a default (5)
X_aug, Y_aug = sm.fit_resample(X, Y)

X_aug = pd.get_dummies(X_aug, columns = ['TIC type'])
X = pd.get_dummies(X, columns = ['TIC type'])
albero4 = albero4.fit(X_aug, Y_aug)
predictions = albero4.predict(X)
print(classification_report(Y, predictions, digits = 3))

              precision    recall  f1-score   support

           0      0.903     0.700     0.789        80
           1      0.467     0.778     0.583        27

    accuracy                          0.720       107
   macro avg      0.685     0.739     0.686       107
weighted avg      0.793     0.720     0.737       107



In [84]:
prob4 = albero4.predict_proba(X)
prob4


array([[0.7       , 0.3       ],
       [0.7       , 0.3       ],
       [1.        , 0.        ],
       [0.76923077, 0.23076923],
       [0.33333333, 0.66666667],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.21428571, 0.78571429],
       [1.        , 0.        ],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.33333333, 0.66666667],
       [0.33333333, 0.66666667],
       [0.7       , 0.3       ],
       [1.        , 0.        ],
       [0.33333333, 0.66666667],
       [0.21428571, 0.78571429],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.33333333, 0.66666667],
       [0.76923077, 0.23076923],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.

## Albero 5 vs Altri

In [90]:
#ora valuto 5 vs altri smote 
#porto k_neighbors a 3 per scarso numero di rappresentanti

X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 5 else 0 for x in dfAlbero['COD ISTO']])

albero5 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'smote', albero5, 5, 50, seed)#ho abbassato momentaneamente il numero di neighbors a 3
prova.fit_score()
prova.print_result()

f1: 0.085145

Sensibilità: 0.054094

Accuratezza: 0.799065

Precisione: 0.200000

Matthews: 0.015901

Kappa di Cohen: 0.012499



In [91]:
#5 vs tutti augmentation con random noise
X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 5 else 0 for x in dfAlbero['COD ISTO']])
albero5 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
prova = SEC_bin(X,Y, 'oversample', albero5, 5, 50, seed)#ho abbassato momentaneamente il numero di neighbors
prova.fit_score()
prova.print_result()

f1: 0.053828

Sensibilità: 0.031022

Accuratezza: 0.667103

Precisione: 0.204000

Matthews: -0.048861

Kappa di Cohen: -0.029582



### Probabilità 5 vs Altri

In [92]:
X = dfAlbero.iloc[:,:-1]
Y = pd.Series([1 if x == 5 else 0 for x in dfAlbero['COD ISTO']])
albero5 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

sm=SMOTENC(random_state=seed, categorical_features=[3], k_neighbors = 4)#k_neighbors abbassato per pochi rappresentanti
X_aug, Y_aug = sm.fit_resample(X, Y)

X_aug = pd.get_dummies(X_aug, columns = ['TIC type'])
X = pd.get_dummies(X, columns = ['TIC type'])
albero5 = albero5.fit(X_aug, Y_aug)
predictions = albero5.predict(X)
print(classification_report(Y, predictions, digits = 3))

              precision    recall  f1-score   support

           0      0.988     0.814     0.892       102
           1      0.174     0.800     0.286         5

    accuracy                          0.813       107
   macro avg      0.581     0.807     0.589       107
weighted avg      0.950     0.813     0.864       107



In [93]:
prob5 = albero5.predict_proba(X)
prob5

array([[1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.0990099 , 0.9009901 ],
       [1.        , 0.        ],
       [0.47368421, 0.52631579],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.0990099 , 0.9009901 ],
       [0.47368421, 0.52631579],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.

In [102]:
#ora devo far confluire le probabilità fornite dai 5 alberi in un unico albero di decisione
#provo, ma non sono sicuro di aver capito bene come dovrei indurre un albero di decisione dalle probabilità


#creo samples e label
Y = dfAlbero['COD ISTO'].copy()
X = pd.DataFrame()

X['Prob1'] = np.transpose(prob1)[1]
X['Prob2'] = np.transpose(prob2)[1]
X['Prob3'] = np.transpose(prob3)[1]
X['Prob4'] = np.transpose(prob4)[1]
X['Prob5'] = np.transpose(prob5)[1]

#alleno sull'intero dataset, senza augmentation
alberoComposto = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)
alberoComposto.fit(X,Y)
predictions1 = alberoComposto.predict(X)
print(classification_report(Y, predictions1, digits = 3))

              precision    recall  f1-score   support

           1      0.667     0.250     0.364        16
           2      0.000     0.000     0.000         5
           3      0.667     0.963     0.788        54
           4      0.810     0.630     0.708        27
           5      1.000     0.400     0.571         5

    accuracy                          0.701       107
   macro avg      0.629     0.449     0.486       107
weighted avg      0.687     0.701     0.657       107



/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/anaconda3/envs/tirocinio/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

## Problema con calcolo accuratezza nel caso dell'augmentation
Se la suddivisione in folder della cross val precede l'augmentation con smote, in molti casi il training set non contiene rappresentanti delle classi minoritarie che devo sottoporre a cross validation

Quindi o ignoro le folder che non contengono un numero opportuno di samples delle classi da augmentare (caso sotto), oppure non faccio la cross validation per misurare l'accuratezza, ma un generico split iterato

In [133]:
#Provo a fare augmentation con SMOTE delle due classi minoritarie, triplicandole
#Valuto accuratezza della cross validation, ignorando le folder che non contengono samples delle classi minoritarie

from sklearn.model_selection import KFold

sm = SMOTE(sampling_strategy = {2:15,5:15}, random_state = seed, k_neighbors = 1)
X_aug, Y_aug = sm.fit_resample(X,Y)
alberoComposto = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

accuracy_median_scores = []
for niter in range(100):
    
    
    skf = KFold(n_splits=5)
    
    accuracy_scores = []

    for train_index, test_index in skf.split(X, Y): 

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]

        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
        
        if  2 not in Y_train.value_counts().index or 5 not in Y_train.value_counts().index or Y_train.value_counts()[2]<=1 or Y_train.value_counts()[5]<=1:
            continue
            
        X_train_aug, Y_train_aug = sm.fit_resample(X_train, Y_train)
        
        alberodummy = albero1.fit(X_train_aug, Y_train_aug)
        
        predictions = alberodummy.predict(X_test)
        
        accuracy_scores.append(accuracy_score(predictions, Y_test))
        
    accuracy_median_scores.append( np.mean(accuracy_scores))

np.mean(accuracy_median_scores)

0.515873015873016

In [135]:
#Valuto accuratezza splittando a caso, senza fare cross val

sm = SMOTE(sampling_strategy = {2:15,5:15}, random_state = seed, k_neighbors = 1)

alberoComposto = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

accuracy_scores = []
for niter in range(50):
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

    if  2 not in Y_train.value_counts().index or 5 not in Y_train.value_counts().index or Y_train.value_counts()[2]<=1 or Y_train.value_counts()[5]<=1:
            continue
    X_train_aug, Y_train_aug = sm.fit_resample(X_train, Y_train)
    alberodummy = alberoComposto.fit(X_train_aug, Y_train_aug)
    predictions = alberodummy.predict(X_test)
    accuracy_scores.append(accuracy_score(predictions, Y_test))
        
    

np.mean(accuracy_scores)

0.487012987012987

In [136]:
#Ora provo a indurre l'albero allenandolo sull'intero ds augmentato (smote che triplica le due classi minoritarie)

sm = SMOTE(sampling_strategy = {2:15,5:15}, random_state = seed, k_neighbors = 1)

alberoComposto = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

X_aug, Y_aug = sm.fit_resample(X,Y)
scores = cross_val_score(alberoComposto, X_aug, Y_aug)
scores.mean()

0.5849230769230769

In [142]:
#ora visuzlizzo l'albero

import graphviz

sm = SMOTE(sampling_strategy = {2:15,5:15}, random_state = seed, k_neighbors = 1)

alberoComposto = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 4, min_samples_leaf = 2)

X_aug, Y_aug = sm.fit_resample(X,Y)
alberoComposto = alberoComposto.fit(X_aug, Y_aug)

data = tree.export_graphviz(alberoComposto, out_file=None, 
                      feature_names=X.columns,  
                      class_names=pd.Series(Y.unique()).astype(str).sort_values().tolist(),  
                      filled=True, rounded=True,  
                      special_characters=True) 
graph = graphviz.Source(data) 
graph.render("alberoComposto")

'alberoComposto.pdf'

# Riassunto

In [146]:
X['COD ISTO'] = Y
X
#Ogni feature rappresenta la probabilità che un sample apprtenga alla rispettiva classe piuttosto che alle altre 4
#secondo quanto ricavato dagli alberi per la classificazione binaria

,Prob1,Prob2,Prob3,Prob4,Prob5,COD ISTO
0,1.000000,0.333333,0.454545,0.300000,0.000000,1
1,1.000000,0.333333,0.571429,0.300000,0.000000,1
2,0.454545,0.000000,0.520000,0.000000,0.000000,1
3,0.348485,0.618182,0.454545,0.230769,0.000000,1
4,0.348485,0.618182,0.571429,0.666667,0.900990,1
...,...,...,...,...,...,...
102,0.047619,0.000000,0.454545,0.230769,0.000000,1
103,0.348485,0.618182,0.571429,0.666667,0.900990,5
104,0.300000,0.000000,0.333333,0.000000,0.526316,5
105,0.348485,0.618182,0.000000,0.666667,0.900990,5
